In [0]:
#código para o desafio do bootcamp de TPD

1) Primeiro passo - Seção Spark

In [0]:
from pyspark.sql import SparkSession #importa a biblioteca que cria a seção do spark

In [0]:
#inicia a seção para a utilização do spark
spark = SparkSession.builder.appName("desafio_tpd").getOrCreate() #cria a seção caso não exista ou obtém a já criada

In [0]:
%fs ls /FileStore/tables    

path,name,size
dbfs:/FileStore/tables/Mall_Customers.csv,Mall_Customers.csv,4286
dbfs:/FileStore/tables/iris_bezdekIris.csv,iris_bezdekIris.csv,4551
dbfs:/FileStore/tables/regressaoLinear-1.csv,regressaoLinear-1.csv,564
dbfs:/FileStore/tables/regressaoLinear.csv,regressaoLinear.csv,564
dbfs:/FileStore/tables/transport_nodes_7c826.csv,transport_nodes_7c826.csv,465
dbfs:/FileStore/tables/transport_relationships_c2bfc.csv,transport_relationships_c2bfc.csv,550
dbfs:/FileStore/tables/u.data,u.data,1979173


In [0]:
diretorioDatasetDiabets="/FileStore/tables/Mall_Customers.csv"  #diretório que contém o arquivo a ser utilizado

In [0]:
data = spark.read.format("csv").options(header="true", inferschema="true").load(diretorioDatasetDiabets)  #realiza a leitura do dataset

In [0]:
data.show(5,False)  #mostrando as 5 primeiras instâncias do dataset

+----------+------+---+------------------+----------------------+
CustomerID|Genre |Age|Annual Income (k$)|Spending Score (1-100)|
+----------+------+---+------------------+----------------------+
1 |Male |19 |15 |39 |
2 |Male |21 |15 |81 |
3 |Female|20 |16 |6 |
4 |Female|23 |16 |77 |
5 |Female|31 |17 |40 |
+----------+------+---+------------------+----------------------+
only showing top 5 rows

In [0]:
data.columns #mostra as colunas do dataset

Out[7]: ['CustomerID', 'Genre', 'Age', 'Annual Income (k$)', 'Spending Score (1-100)']

In [0]:
data.printSchema()  #mostra o esquema inferido pelas variáveis

root
-- CustomerID: integer (nullable = true)
-- Genre: string (nullable = true)
-- Age: integer (nullable = true)
-- Annual Income (k$): integer (nullable = true)
-- Spending Score (1-100): integer (nullable = true)

In [0]:
data.shape()

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-2464364679658479> in <module> 
 ----> 1 data . shape ( ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in __getattr__ (self, name) 
 1332 if name not in self . columns : 
 1333 raise AttributeError(
 -> 1334 "'%s' object has no attribute '%s'" % (self.__class__.__name__, name))
 1335 jc = self . _jdf . apply ( name ) 
 1336 return Column ( jc ) 

 AttributeError : 'DataFrame' object has no attribute 'shape'

In [0]:
data.count()  #conta a quantidade de registros

Out[9]: 200

In [0]:
#verificando a existência ou não de valores nulos
from pyspark.sql.functions import isnan, when, count, col

data.select([count(when(isnan(c), c)).alias(c) for c in data.columns]).show()

+----------+-----+---+------------------+----------------------+
CustomerID|Genre|Age|Annual Income (k$)|Spending Score (1-100)|
+----------+-----+---+------------------+----------------------+
 0| 0| 0| 0| 0|
+----------+-----+---+------------------+----------------------+

In [0]:
data.describe().show() #mostrando as estatísticas do dataset

+-------+------------------+------+-----------------+------------------+----------------------+
summary| CustomerID| Genre| Age|Annual Income (k$)|Spending Score (1-100)|
+-------+------------------+------+-----------------+------------------+----------------------+
 count| 200| 200| 200| 200| 200|
 mean| 100.5| null| 38.85| 60.56| 50.2|
 stddev|57.879184513951124| null|13.96900733155888| 26.26472116527124| 25.823521668370173|
 min| 1|Female| 18| 15| 1|
 max| 200| Male| 70| 137| 99|
+-------+------------------+------+-----------------+------------------+----------------------+

Verificando se existem outliers (anomalias) nos dados. Para isso, será utilizado o boxplot.

In [0]:
#selecionando apenas algumas colunas
data_outliers=data.select([c for c in data.columns if c in ['Age','Annual Income (k$)','Spending Score (1-100)']])
data_outliers.show()

+---+------------------+----------------------+
Age|Annual Income (k$)|Spending Score (1-100)|
+---+------------------+----------------------+
 19| 15| 39|
 21| 15| 81|
 20| 16| 6|
 23| 16| 77|
 31| 17| 40|
 22| 17| 76|
 35| 18| 6|
 23| 18| 94|
 64| 19| 3|
 30| 19| 72|
 67| 19| 14|
 35| 19| 99|
 58| 20| 15|
 24| 20| 77|
 37| 20| 13|
 22| 20| 79|
 35| 21| 35|
 20| 21| 66|
 52| 23| 29|
 35| 23| 98|
+---+------------------+----------------------+
only showing top 20 rows

In [0]:
#boxplot
display(data_outliers)

Age,Annual Income (k$),Spending Score (1-100)
19,15,39
21,15,81
20,16,6
23,16,77
31,17,40
22,17,76
35,18,6
23,18,94
64,19,3
30,19,72


In [0]:
#histograma
display(data_outliers)

Age,Annual Income (k$),Spending Score (1-100)
19,15,39
21,15,81
20,16,6
23,16,77
31,17,40
22,17,76
35,18,6
23,18,94
64,19,3
30,19,72


Conhecendo o dataset

In [0]:
display(data.select(["Genre","Annual Income (k$)"]))

--------------------------------------------------------------------------- 
 Exception Traceback (most recent call last)
 <command-2464364679658454> in <module> 
 ----> 1 display ( data . select ( [ "Genre" , "Annual Income (k$)" ] ) . groupBy ( [ "Genre" ] ) ) 

 /local_disk0/tmp/1619378089366-0/PythonShell.py in display (self, input, *args, **kwargs) 
 1107 self . displayHTML ( input . _repr_html_ ( ) ) 
 1108 else : 
 -> 1109 raise Exception ( genericErrorMsg ) 
 1110 
 1111 def displayHTML ( self , html ) : 

 Exception : Cannot call display(<class 'pyspark.sql.group.GroupedData'>)
 Call help(display) for more info.

In [0]:
display(data.groupBy("Genre").avg("Annual Income (k$)"))

Genre,avg(Annual Income (k$))
Female,59.25
Male,62.22727272727273


In [0]:
#distribuição dos consumidores pela idade
display(data.select(["Age"]))

Age
19
21
20
23
31
22
35
23
64
30


In [0]:
#encontrando a distribuição da idade x sexo
display(data.select(["Age","Genre"]))

Age,Genre
19,Male
21,Male
20,Female
23,Female
31,Female
22,Female
35,Female
23,Female
64,Male
30,Female


In [0]:
#encontrando a distribuição da idade x sexo
display(data.select(["Age","Genre"]))

Age,Genre
19,Male
21,Male
20,Female
23,Female
31,Female
22,Female
35,Female
23,Female
64,Male
30,Female


A variável "Spending Score (1-100)" ou pontuação de consumo, corresponde à nota atribuída pelo shopping aos consumidores. Quanto mais próxima de 100 indica que o consumidor é mais "lucrativo" para o shopping.

In [0]:
#encontrando as notas atribuidas por gênero 
display(data.select(["Spending Score (1-100)","Genre","Annual Income (k$)"]))

Spending Score (1-100),Genre,Annual Income (k$)
39,Male,15
81,Male,15
6,Female,16
77,Female,16
40,Female,17
76,Female,17
6,Female,18
94,Female,18
3,Male,19
72,Female,19


Analisando a correlação entre Idade e Nota Atribuída

In [0]:
#Analisando as correlações
from pyspark.mllib.stat import Statistics

In [0]:
rdd_1=data.select("Spending Score (1-100)").rdd.flatMap(lambda x:x)  #utilizada para transforma das colunas do dataframe em colunas para serem empregadas na análise de correlaçoes
rdd_2=data.select("Age").rdd.flatMap(lambda x:x)

In [0]:
Statistics.corr(rdd_1,rdd_2,method="pearson")  #definindo a coeficiente de correlação de pearson 

Out[22]: -0.32722684603908925

In [0]:
data.stat.corr("Age","Annual Income (k$)",method="pearson")

Out[23]: -0.01239804273606026

Analisando a correlação por gênero

In [0]:
#Selecionando os homens
homens=data[data.Genre=='Male']
homens.show()

+----------+-----+---+------------------+----------------------+
CustomerID|Genre|Age|Annual Income (k$)|Spending Score (1-100)|
+----------+-----+---+------------------+----------------------+
 1| Male| 19| 15| 39|
 2| Male| 21| 15| 81|
 9| Male| 64| 19| 3|
 11| Male| 67| 19| 14|
 15| Male| 37| 20| 13|
 16| Male| 22| 20| 79|
 18| Male| 20| 21| 66|
 19| Male| 52| 23| 29|
 21| Male| 35| 24| 35|
 22| Male| 25| 24| 73|
 24| Male| 31| 25| 73|
 26| Male| 29| 28| 82|
 28| Male| 35| 28| 61|
 31| Male| 60| 30| 4|
 33| Male| 53| 33| 4|
 34| Male| 18| 33| 92|
 42| Male| 24| 38| 92|
 43| Male| 48| 39| 36|
 52| Male| 33| 42| 60|
 54| Male| 59| 43| 60|
+----------+-----+---+------------------+----------------------+
only showing top 20 rows

In [0]:
#selecionando mulheres
mulheres=data[data.Genre=='Female']
mulheres.show()

+----------+------+---+------------------+----------------------+
CustomerID| Genre|Age|Annual Income (k$)|Spending Score (1-100)|
+----------+------+---+------------------+----------------------+
 3|Female| 20| 16| 6|
 4|Female| 23| 16| 77|
 5|Female| 31| 17| 40|
 6|Female| 22| 17| 76|
 7|Female| 35| 18| 6|
 8|Female| 23| 18| 94|
 10|Female| 30| 19| 72|
 12|Female| 35| 19| 99|
 13|Female| 58| 20| 15|
 14|Female| 24| 20| 77|
 17|Female| 35| 21| 35|
 20|Female| 35| 23| 98|
 23|Female| 46| 25| 5|
 25|Female| 54| 28| 14|
 27|Female| 45| 28| 32|
 29|Female| 40| 29| 31|
 30|Female| 23| 29| 87|
 32|Female| 21| 30| 73|
 35|Female| 49| 33| 14|
 36|Female| 21| 33| 81|
+----------+------+---+------------------+----------------------+
only showing top 20 rows

In [0]:
#correlação entre Idade e o Salário Anual - Homens
homens.stat.corr("Age","Spending Score (1-100)",method="pearson")

Out[26]: -0.27962764627493125

In [0]:
#correlação entre Idade e o Salário Anual - Homens
mulheres.stat.corr("Age","Spending Score (1-100)",method="pearson")

Out[27]: -0.3754385837780473

Aplicando o Kmeans

In [0]:
from pyspark.ml.evaluation import ClusteringEvaluator  #biblioteca utilizada para a avaliação em cada um dos clusters
from pyspark.ml.clustering import KMeans #biblioteca utilizada para a criação do modelo de clusterização utilizando o K-means

In [0]:
from pyspark.ml.feature import VectorAssembler  #transformando os dados em vetores de características

vecAssembler = VectorAssembler(inputCols=["Spending Score (1-100)","Annual Income (k$)"], outputCol="features") #utilizada para transformar os dados em um vetor (define o objeto)
new_df = vecAssembler.transform(data) #Aplico a transformação
new_df.show()

+----------+------+---+------------------+----------------------+-----------+
CustomerID| Genre|Age|Annual Income (k$)|Spending Score (1-100)| features|
+----------+------+---+------------------+----------------------+-----------+
 1| Male| 19| 15| 39|[39.0,15.0]|
 2| Male| 21| 15| 81|[81.0,15.0]|
 3|Female| 20| 16| 6| [6.0,16.0]|
 4|Female| 23| 16| 77|[77.0,16.0]|
 5|Female| 31| 17| 40|[40.0,17.0]|
 6|Female| 22| 17| 76|[76.0,17.0]|
 7|Female| 35| 18| 6| [6.0,18.0]|
 8|Female| 23| 18| 94|[94.0,18.0]|
 9| Male| 64| 19| 3| [3.0,19.0]|
 10|Female| 30| 19| 72|[72.0,19.0]|
 11| Male| 67| 19| 14|[14.0,19.0]|
 12|Female| 35| 19| 99|[99.0,19.0]|
 13|Female| 58| 20| 15|[15.0,20.0]|
 14|Female| 24| 20| 77|[77.0,20.0]|
 15| Male| 37| 20| 13|[13.0,20.0]|
 16| Male| 22| 20| 79|[79.0,20.0]|
 17|Female| 35| 21| 35|[35.0,21.0]|
 18| Male| 20| 21| 66|[66.0,21.0]|
 19| Male| 52| 23| 29|[29.0,23.0]|
 20|Female| 35| 23| 98|[98.0,23.0]|
+----------+------+---+------------------+----------------------+-----------+
only showing top 20 rows

In [0]:
#aplica o processo de clusterização
kmeans = KMeans(k=3, seed=1)  # declara o objeto - 3 clusters 
model = kmeans.fit(new_df.select('features')) #aplica o treinamento

In [0]:
#cria o dataset com a indicação sobre qual cluster cada conjunto de dados foi adicionado
df_final = model.transform(new_df)
df_final.show() 

+----------+------+---+------------------+----------------------+-----------+----------+
CustomerID| Genre|Age|Annual Income (k$)|Spending Score (1-100)| features|prediction|
+----------+------+---+------------------+----------------------+-----------+----------+
 1| Male| 19| 15| 39|[39.0,15.0]| 1|
 2| Male| 21| 15| 81|[81.0,15.0]| 1|
 3|Female| 20| 16| 6| [6.0,16.0]| 1|
 4|Female| 23| 16| 77|[77.0,16.0]| 1|
 5|Female| 31| 17| 40|[40.0,17.0]| 1|
 6|Female| 22| 17| 76|[76.0,17.0]| 1|
 7|Female| 35| 18| 6| [6.0,18.0]| 1|
 8|Female| 23| 18| 94|[94.0,18.0]| 1|
 9| Male| 64| 19| 3| [3.0,19.0]| 1|
 10|Female| 30| 19| 72|[72.0,19.0]| 1|
 11| Male| 67| 19| 14|[14.0,19.0]| 1|
 12|Female| 35| 19| 99|[99.0,19.0]| 1|
 13|Female| 58| 20| 15|[15.0,20.0]| 1|
 14|Female| 24| 20| 77|[77.0,20.0]| 1|
 15| Male| 37| 20| 13|[13.0,20.0]| 1|
 16| Male| 22| 20| 79|[79.0,20.0]| 1|
 17|Female| 35| 21| 35|[35.0,21.0]| 1|
 18| Male| 20| 21| 66|[66.0,21.0]| 1|
 19| Male| 52| 23| 29|[29.0,23.0]| 1|
 20|Female| 35| 23| 98|[98.0,23.0]| 1|
+----------+------+---+------------------+----------------------+-----------+----------+
only showing top 20 rows

In [0]:
data.describe().show()

+-------+------------------+------+-----------------+------------------+----------------------+
summary| CustomerID| Genre| Age|Annual Income (k$)|Spending Score (1-100)|
+-------+------------------+------+-----------------+------------------+----------------------+
 count| 200| 200| 200| 200| 200|
 mean| 100.5| null| 38.85| 60.56| 50.2|
 stddev|57.879184513951124| null|13.96900733155888| 26.26472116527124| 25.823521668370173|
 min| 1|Female| 18| 15| 1|
 max| 200| Male| 70| 137| 99|
+-------+------------------+------+-----------------+------------------+----------------------+

In [0]:
df_final.groupby('prediction').count().show()

+----------+-----+
prediction|count|
+----------+-----+
 1| 123|
 2| 39|
 0| 38|
+----------+-----+

In [0]:
display(df_final)

CustomerID,Genre,Age,Annual Income (k$),Spending Score (1-100),features,prediction
1,Male,19,15,39,"List(1, 2, List(), List(39.0, 15.0))",1
2,Male,21,15,81,"List(1, 2, List(), List(81.0, 15.0))",1
3,Female,20,16,6,"List(1, 2, List(), List(6.0, 16.0))",1
4,Female,23,16,77,"List(1, 2, List(), List(77.0, 16.0))",1
5,Female,31,17,40,"List(1, 2, List(), List(40.0, 17.0))",1
6,Female,22,17,76,"List(1, 2, List(), List(76.0, 17.0))",1
7,Female,35,18,6,"List(1, 2, List(), List(6.0, 18.0))",1
8,Female,23,18,94,"List(1, 2, List(), List(94.0, 18.0))",1
9,Male,64,19,3,"List(1, 2, List(), List(3.0, 19.0))",1
10,Female,30,19,72,"List(1, 2, List(), List(72.0, 19.0))",1


In [0]:
#transformando os dados utilizando o pandas 
data_pandas = df_final.toPandas().set_index('CustomerID')
data_pandas.head()

,Genre,Age,Annual Income (k$),Spending Score (1-100),features,prediction
CustomerID,,,,,,
1,Male,19,15,39,"[39.0, 15.0]",4
2,Male,21,15,81,"[81.0, 15.0]",1
3,Female,20,16,6,"[6.0, 16.0]",4
4,Female,23,16,77,"[77.0, 16.0]",1
5,Female,31,17,40,"[40.0, 17.0]",4


In [0]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,10))
plt.scatter(data_pandas['Annual Income (k$)'], data_pandas['Spending Score (1-100)'], c=data_pandas.prediction)
plt.xlabel('Annual Income (k$)')
plt.ylabel('Spending Score (1-100)')
plt.show()